In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# import gymnasium as gym
import gym
# from gymnasium import Env, spaces
from gym import Env, spaces
import pandas as pd
import random
import os

import torch
from torch import nn
from torch.optim import AdamW
import torch.nn.functional as F
from tqdm import tqdm



In [ ]:
PATH = "drive/MyDrive/Trading_bot/"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
INITIAL_BALANCE = 10000

class TradingEnv(Env):
  def __init__(self, data_path, initial_step=0, end_step=0, random_steps=False):
    #self.reset() maybe
    self.df = self.__read_df(data_path)
    self.action_space = spaces.Discrete(3)
    # self.action_space = spaces.Tuple((spaces.Discrete(3),
    #                                  spaces.Box(low=0, high=1, shape=(1,)))) # Buy, Sell, do Nothing
    self.observation_space = spaces.Box(low=0, high=100000, shape=(6, 6), dtype=np.float32)
    self.current_step = initial_step
    self.end_step = self.__clamp(end_step)
    self.random_steps = random_steps



  def __read_df(self, data_path):
    df = pd.read_csv(data_path, quotechar='"')
    # df = df.iloc[::-1]
    # self.df['Date'] = pd.to_datetime(self.df['Date'])
    # df[['Volume']] = df[['Volume']].applymap(lambda value: float(value.replace(",", "")))
    return df

  def reset(self):
    self.shares_held = 0
    self.balance = INITIAL_BALANCE
    self.current_step = 6
    if self.random_steps:
      self.current_step = random.randint(6, self.df.shape[0] - 1)
      self.end_step = random.randint(self.current_step, len(self.df.loc[:, 'Open'].values - 1))
    self.net_worth = INITIAL_BALANCE
    self.shares_sold = 0
    self.total_shares_sold_value = 0.0
    self.last_price = random.uniform(
          self.df.loc[self.current_step, 'Open'],
          self.df.loc[self.current_step, 'Close']
      )
    return self.__getNextObs()

  def render(self):
    # print(f"balance : {self.balance}, portfolio : {self.shares_held * self.last_price}, net_worth : {self.net_worth}")
    print(f"net_worth : {self.net_worth}")

  def step(self, action):
    prev_net_worth = self.net_worth
    self.__take_action(action)
    self.current_step += 1
    if self.current_step > len(self.df.loc[:, 'Open'].values) - 1:
        self.current_step = 6
    returns = (self.net_worth - prev_net_worth) / prev_net_worth
    reward = self.__compute_reward(action, prev_net_worth) # TODO : change to action[0] after stable baselines
    done = self.net_worth <= 0 or self.current_step == self.end_step
    obs = self.__getNextObs()
    return obs, reward, done, {}

  def __getNextObs(self):
    frame = np.array([
        self.df.iloc[self.current_step - 6: self.current_step, self.df.columns.get_loc('Open')],
        self.df.iloc[self.current_step - 6: self.current_step, self.df.columns.get_loc('High')],
        self.df.iloc[self.current_step - 6: self.current_step, self.df.columns.get_loc('Low')],
        self.df.iloc[self.current_step - 6: self.current_step, self.df.columns.get_loc('Close')],
        self.df.iloc[self.current_step - 6: self.current_step, self.df.columns.get_loc('Volume')]
    ])
    obs = np.append(frame, [[
          self.balance,
          self.net_worth,
          self.shares_held,
          self.shares_sold,
          self.last_price,
          self.total_shares_sold_value
      ]], axis=0)
    return obs

  def __compute_reward(self, action, prev_net_worth):
    returns = (self.net_worth - prev_net_worth) / prev_net_worth
    reward = 0.0
    if action != 2:
      #penalize the bot for making a trade
      reward -= .0002
    reward += returns
    if returns == 0.0:
      reward -= .0005
    return reward
  def __take_action(self, action):
    #select a price randomly
    current_price = random.uniform(
          self.df.loc[self.current_step, 'Open'],
          self.df.loc[self.current_step, 'Close']
      )
    self.last_price = current_price
    # action, amount = action # here amount refers to the percentage of the account balance
    amount = .2
    if action == 0:
      #buy an amount of shares
      quantity = amount * self.balance / current_price # how many oz of gold for example
      self.shares_held += quantity
      self.balance -= amount * self.balance
    elif action == 1:
      #Sell
      quantity = amount * self.shares_held
      self.shares_held -= quantity
      self.shares_sold += quantity
      self.total_shares_sold_value += quantity * current_price
      self.balance += quantity * current_price
    self.net_worth = self.balance + self.shares_held * current_price
  def __clamp(self, end_step):
    if self.df.shape[0] + 6 < end_step < self.df.shape[0] - 1:
      return end_step
    return self.df.shape[0] - 1

In [ ]:
TSLA_path = os.path.join(PATH, "data","TSLA.csv")
MSFT_path = os.path.join(PATH, "data", "MSFT.csv")
AAPL_path = os.path.join(PATH, "data", "AAPL.csv")

In [ ]:
#creating the training environment of different stocks
#The training set is the first 3 years
#The validation set is the last 2 years
TSLA_training_env = TradingEnv(TSLA_path, initial_step=0, end_step=837)
MSFT_training_env = TradingEnv(MSFT_path, initial_step=0, end_step=837)
AAPL_training_env = TradingEnv(TSLA_path, initial_step=0, end_step=837)

In [ ]:
print(TSLA_training_env.reset())

[[2.00700000e+01 2.11480010e+01 2.05499990e+01 2.24979990e+01
  2.18926680e+01 2.21693330e+01]
 [2.02960000e+01 2.14000000e+01 2.26600000e+01 2.31439990e+01
  2.25266670e+01 2.27999990e+01]
 [1.90486680e+01 2.00673330e+01 2.04433330e+01 2.17666660e+01
  2.14839990e+01 2.19400010e+01]
 [1.92333340e+01 2.09906670e+01 2.20599990e+01 2.23233340e+01
  2.19933340e+01 2.24880010e+01]
 [3.00874500e+08 3.12610500e+08 4.11382500e+08 2.17290000e+08
  1.36900500e+08 1.14364500e+08]
 [1.00000000e+04 1.00000000e+04 0.00000000e+00 0.00000000e+00
  2.28777009e+01 0.00000000e+00]]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
TSLA_training_env.df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-10-24,20.070000,20.296000,19.048668,19.233334,19.233334,300874500
1,2018-10-25,21.148001,21.400000,20.067333,20.990667,20.990667,312610500
2,2018-10-26,20.549999,22.660000,20.443333,22.059999,22.059999,411382500
3,2018-10-29,22.497999,23.143999,21.766666,22.323334,22.323334,217290000
4,2018-10-30,21.892668,22.526667,21.483999,21.993334,21.993334,136900500


In [ ]:
dims = MSFT_training_env.observation_space.shape[0]
actions = MSFT_training_env.action_space.n


In [ ]:
#code from https://github.com/Touutae-lab/RL-Breeze
class A2C:
  def __init__(self, dims, actions):
    self.actor = nn.Sequential(
        nn.Linear(dims, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, actions),
        nn.Softmax(dim=-1)
    )
    self.critic = critic = nn.Sequential(
        nn.Linear(dims, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 1)
    )

  def learn(self, env, episodes, alpha=1e-4, gamma=.99, device='cuda'):
      actor_optim = torch.optim.AdamW(self.actor.parameters(), lr=1e-3)
      critic_optim = torch.optim.AdamW(self.critic.parameters(), lr=1e-4)
      stats = {'Actor Loss': [], 'Critic Loss': [], 'Returns': []}

      self.actor.to(device)
      self.critic.to(device)

      for episode in range(1, episodes + 1):
          state = torch.Tensor(env.reset()).to(device)
          done = False
          ep_return = torch.zeros(1, 1).to(device)
          I = 1

          while not done:
              action = self.actor(state).multinomial(1).detach()
              next_state, reward, done, _ = env.step(action[0])
              next_state = torch.Tensor(next_state).to(device)
              value = self.critic(state)
              target = reward + ~done * gamma * self.critic(next_state).detach()
              critic_loss = F.mse_loss(value, target)
              self.critic.zero_grad()
              critic_loss.backward()
              critic_optim.step()

              advantage = (target - value).detach()
              probs = self.actor(state)
              log_probs = torch.log(probs + 1e-6)
              action_log_prob = log_probs.gather(1, action)
              entropy = -torch.sum(probs * log_probs, dim=-1, keepdim=True)
              actor_loss = -I * action_log_prob * advantage - 0.01 * entropy
              actor_loss = actor_loss.mean()
              self.actor.zero_grad()
              actor_loss.backward()
              actor_optim.step()

              ep_return += reward
              state = next_state
              I = I * gamma
          # Record training performance
          stats['Actor Loss'].append(actor_loss.item())
          stats['Critic Loss'].append(critic_loss.item())
          stats['Returns'].append(ep_return.mean().item())
      return stats
  def save(self, actor_path='actor_model_minus.pth',critic_path='critic_model_minus.pth'):
        torch.save(self.actor.state_dict(), actor_path)
        torch.save(self.critic.state_dict(), critic_path)



In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
model = A2C(dims, actions)
stats = model.learn(MSFT_training_env, 50)
stats = model.learn(AAPL_training_env, 50)
stats = model.learn(TSLA_training_env, 50)
#saving the model
model.save()